<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Creer un dashboard PowerBI

Ce Notebook permet de transformer des fichiers FEC de votre entreprise en un tableau de bord Microsoft Power BI.
Le FEC (fichier des écritures comptables) est un export standard des logiciels de comptabilite et une obligation légale en france depuis 2014 afin de déposer ses comptes de manière electronique auprès des services fiscaux.
<br><br>
-Durée de l’installation = 5 minutes<br>
-Support d’installation = [Page Notion](https://www.notion.so/Mode-d-emploi-FECthis-7fc142f2d7ae4a3889fbca28a83acba2/)<br>
-Niveau = Facile<br>

**Author:** [Alexandre STEVENS](https://www.linkedin.com/in/alexandrestevenspbix/)

**Tag:** #powerbi #dashboard #tableaudebord #FEC #comptabilite #accounting 

## Input

### Librairie

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import os
import re
import naas

### Variables à modifier par l'utilisateur

In [2]:
# Entreprise
LOGO = "https://landen.imgix.net/e5hx7wyzf53f/assets/26u7xg7u.png?w=400"
COLOR_1 = "#CCCCCC"
COLOR_2 = "#999999"

### Lire les fichiers FEC

In [3]:
def get_all_fec(file_regex,
                sep=",",
                decimal=".",
                encoding=None,
                header=None,
                usecols=None,
                names=None,
                dtype=None):
    # Create df init
    df = pd.DataFrame()

    # Get all files in INPUT_FOLDER
    files = [f for f in os.listdir() if re.search(file_regex, f)]
    if len(files) == 0:
        print(f"Aucun fichier FEC ne correspond au standard de nomination")
    else:
        for file in files:
            # Open file and create df
            print(file)
            tmp_df = pd.read_csv(file,
                                 sep=sep,
                                 decimal=decimal,
                                 encoding=encoding,
                                 header=header,
                                 usecols=usecols,
                                 names=names,
                                 dtype=dtype)
            # Add filename to df
            tmp_df['NOM_FICHIER'] = file

            # Concat df
            df = pd.concat([df, tmp_df], axis=0, sort=False)
    return df

In [4]:
file_regex = "^\d{9}FEC\d{8}.txt"

db_init = get_all_fec(file_regex,
                      sep='\t',
                      decimal=',',
                      encoding='ISO-8859-1',
                      header=0)
db_init

000000000FEC20181231.txt
000000000FEC20171231.txt


,JournalCode,JournalLib,EcritureNum,EcritureDate,CompteNum,CompteLib,CompAuxNum,CompAuxLib,PieceRef,PieceDate,...,DateLet,ValidDate,Montantdevise,Idevise,DateRglt,ModeRglt,NatOp,IdClient,Unnamed: 22,NOM_FICHIER
0,AN,A nouveaux,AN0000001,20180101,20500000,"BREVETS, LICENCES, LOGICIELS..",,,1,20180101,...,NaN,20190326,,EUR,,,,,NaN,000000000FEC20181231.txt
1,AN,A nouveaux,AN0000001,20180101,20700000,FONDS COMMERCIAL,,,1,20180101,...,NaN,20190326,,EUR,,,,,NaN,000000000FEC20181231.txt
2,AN,A nouveaux,AN0000001,20180101,21540000,MATÉRIEL INDUSTRIEL,,,1,20180101,...,NaN,20190326,,EUR,,,,,NaN,000000000FEC20181231.txt
3,AN,A nouveaux,AN0000001,20180101,21810000,AMÉNAG. AGENCEMENTS LOCAUX,,,1,20180101,...,NaN,20190326,,EUR,,,,,NaN,000000000FEC20181231.txt
4,AN,A nouveaux,AN0000001,20180101,21830000,MATÉRIEL BUREAU ET INFORMATIQU,,,1,20180101,...,NaN,20190326,,EUR,,,,,NaN,000000000FEC20181231.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,V1,VENTES,V10000011,20171221,411E010000,CLIENT,,,20170139,20171221,...,NaN,20181024,,EUR,,,,,NaN,000000000FEC20171231.txt
1958,V1,VENTES,V10000011,20171221,411E010000,CLIENT,,,20170139,20171221,...,NaN,20181024,,EUR,,,,,NaN,000000000FEC20171231.txt
1959,V1,VENTES,V10000011,20171221,411E010000,CLIENT,,,20170139,20171221,...,NaN,20181024,,EUR,,,,,NaN,000000000FEC20171231.txt
1960,V1,VENTES,V10000011,20171221,411E010000,CLIENT,,,20170139,20171221,...,NaN,20181024,,EUR,,,,,NaN,000000000FEC20171231.txt


## Model

### Base de donnée FEC

#### Nettoyage des données

In [5]:
db_clean = db_init.copy()

# Selection des colonnes à conserver
to_select = ['NOM_FICHIER',
             'EcritureDate',
             'CompteNum',
             'CompteLib',
             'EcritureLib',
             'Debit',
             'Credit']
db_clean = db_clean[to_select]

# Renommage des colonnes
to_rename = {'EcritureDate': "DATE", 
             'CompteNum': "COMPTE_NUM", 
             'CompteLib': "RUBRIQUE_N3", 
             'EcritureLib': "RUBRIQUE_N4", 
             'Debit': "DEBIT", 
             'Credit': "CREDIT"  }
db_clean = db_clean.rename(columns=to_rename)

#suppression des espaces colonne "COMPTE_NUM"
db_clean["COMPTE_NUM"] = db_clean["COMPTE_NUM"].str.strip()

# Mise au format des colonnes
db_clean = db_clean.astype({"NOM_FICHIER" : str,
                            "DATE" : str,
                            "COMPTE_NUM" : str,
                            "RUBRIQUE_N3" : str,
                            "RUBRIQUE_N4" : str,
                            "DEBIT" : float,
                            "CREDIT" : float,
                            })

# Mise au format colonne date
db_clean["DATE"] = pd.to_datetime(db_clean["DATE"])

db_clean.head(5)

,NOM_FICHIER,DATE,COMPTE_NUM,RUBRIQUE_N3,RUBRIQUE_N4,DEBIT,CREDIT
0,000000000FEC20181231.txt,2018-01-01,20500000,"BREVETS, LICENCES, LOGICIELS..",A Nouveau,4006.60,0.0
1,000000000FEC20181231.txt,2018-01-01,20700000,FONDS COMMERCIAL,A Nouveau,68415.00,0.0
2,000000000FEC20181231.txt,2018-01-01,21540000,MATÉRIEL INDUSTRIEL,A Nouveau,49815.13,0.0
3,000000000FEC20181231.txt,2018-01-01,21810000,AMÉNAG. AGENCEMENTS LOCAUX,A Nouveau,4767.25,0.0
4,000000000FEC20181231.txt,2018-01-01,21830000,MATÉRIEL BUREAU ET INFORMATIQU,A Nouveau,5525.66,0.0


#### Enrichissement de la base

In [6]:
db_enr = db_clean.copy()

# Ajout colonnes entité et période
db_enr['ENTITY'] = db_enr['NOM_FICHIER'].str[:9]
db_enr['PERIOD'] = db_enr['NOM_FICHIER'].str[12:-6]
db_enr['PERIOD'] = pd.to_datetime(db_enr['PERIOD'], format='%Y%m')
db_enr['PERIOD'] = db_enr['PERIOD'].dt.strftime("%Y-%m")

# Ajout colonne month et month_index
db_enr['MONTH'] = db_enr['DATE'].dt.strftime("%b")
db_enr['MONTH_INDEX'] = db_enr['DATE'].dt.month

# Calcul de la valeur debit-crédit
db_enr["VALUE"] = (db_enr["DEBIT"]) - (db_enr["CREDIT"])

db_enr.head(5)

,NOM_FICHIER,DATE,COMPTE_NUM,RUBRIQUE_N3,RUBRIQUE_N4,DEBIT,CREDIT,ENTITY,PERIOD,MONTH,MONTH_INDEX,VALUE
0,000000000FEC20181231.txt,2018-01-01,20500000,"BREVETS, LICENCES, LOGICIELS..",A Nouveau,4006.60,0.0,000000000,2018-12,Jan,1,4006.60
1,000000000FEC20181231.txt,2018-01-01,20700000,FONDS COMMERCIAL,A Nouveau,68415.00,0.0,000000000,2018-12,Jan,1,68415.00
2,000000000FEC20181231.txt,2018-01-01,21540000,MATÉRIEL INDUSTRIEL,A Nouveau,49815.13,0.0,000000000,2018-12,Jan,1,49815.13
3,000000000FEC20181231.txt,2018-01-01,21810000,AMÉNAG. AGENCEMENTS LOCAUX,A Nouveau,4767.25,0.0,000000000,2018-12,Jan,1,4767.25
4,000000000FEC20181231.txt,2018-01-01,21830000,MATÉRIEL BUREAU ET INFORMATIQU,A Nouveau,5525.66,0.0,000000000,2018-12,Jan,1,5525.66


In [7]:
# Calcul résultat pour équilibrage bilan dans capitaux propre
db_rn = db_enr.copy()

db_rn = db_rn[db_rn['COMPTE_NUM'].str.contains(r'^6|^7')]

to_group = ["ENTITY", "PERIOD"]
to_agg = {"VALUE": "sum"}
db_rn = db_rn.groupby(to_group, as_index=False).agg(to_agg)

db_rn ["COMPTE_NUM"] = "10999999"
db_rn ["RUBRIQUE_N3"] = "RESULTAT"

# Reorganisation colonne
to_select = ['ENTITY',
             'PERIOD',
             'COMPTE_NUM',
             'RUBRIQUE_N3',
             'VALUE']
db_rn = db_rn[to_select]
db_rn

,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N3,VALUE
0,000000000,2017-12,10999999,RESULTAT,-81496.79
1,000000000,2018-12,10999999,RESULTAT,-32807.85


### Base de données FEC aggrégée avec variation

#### Aggrégation RUBRIQUE N3

In [8]:
# Calcul var v = création de dataset avec Period_comp pour merge
db_var = db_enr.copy()

# Regroupement 
to_group = ["ENTITY",
            "PERIOD",
            "COMPTE_NUM",
            "RUBRIQUE_N3"]
to_agg = {"VALUE": "sum"}
db_var = db_var.groupby(to_group, as_index=False).agg(to_agg)

# Ajout des résultats au dataframe
db_var = pd.concat([db_var, db_rn], axis=0, sort=False)

# Creation colonne COMP
db_var['PERIOD_COMP'] = (db_var['PERIOD'].str[:4].astype(int) - 1).astype(str) + db_var['PERIOD'].str[-3:]

db_var

,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N3,VALUE,PERIOD_COMP
0,000000000,2017-12,10130000,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.00,2016-12
1,000000000,2017-12,10611000,RESERVE LEGALE,-600.00,2016-12
2,000000000,2017-12,10688000,AUTRES RESERVES,-130673.92,2016-12
3,000000000,2017-12,12000000,RÉSULTAT DE L'EXERCICE BÉNÉFIC,0.00,2016-12
4,000000000,2017-12,20500000,"BREVETS, LICENCES, LOGICIELS..",4006.60,2016-12
...,...,...,...,...,...,...
291,000000000,2018-12,77180000,AUTRES PROD EXCEP S/OP GESTION,-51.00,2017-12
292,000000000,2018-12,79100000,TRANSFERT DE CHARGES D'EXPLOIT,-46.07,2017-12
293,000000000,2018-12,79101000,RÉINT. UTILIS. PERSONNELLE,-1750.50,2017-12
0,000000000,2017-12,10999999,RESULTAT,-81496.79,2016-12


#### Création de la base comparable

In [9]:
db_comp = db_var.copy()

# Suppression de la colonne période
db_comp = db_comp.drop("PERIOD_COMP", axis=1)

# Renommage des colonnes
to_rename = {'VALUE': "VALUE_N-1", 
             'PERIOD': "PERIOD_COMP"}
db_comp = db_comp.rename(columns=to_rename)

db_comp.head(5)

,ENTITY,PERIOD_COMP,COMPTE_NUM,RUBRIQUE_N3,VALUE_N-1
0,000000000,2017-12,10130000,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.00
1,000000000,2017-12,10611000,RESERVE LEGALE,-600.00
2,000000000,2017-12,10688000,AUTRES RESERVES,-130673.92
3,000000000,2017-12,12000000,RÉSULTAT DE L'EXERCICE BÉNÉFIC,0.00
4,000000000,2017-12,20500000,"BREVETS, LICENCES, LOGICIELS..",4006.60


#### Jointure des 2 tables et calcul des variations

In [10]:
# Jointure entre les 2 tables
join_on = ["ENTITY",
           "PERIOD_COMP",
           "COMPTE_NUM",
           "RUBRIQUE_N3"]
db_var = pd.merge(db_var, db_comp, how='left', on=join_on).drop("PERIOD_COMP", axis=1).fillna(0)

#Création colonne Var V
db_var["VARV"] = db_var["VALUE"] - db_var["VALUE_N-1"]

#Création colonne Var P (%)
db_var["VARP"] = db_var["VARV"] / db_var["VALUE_N-1"]

db_var

,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N3,VALUE,VALUE_N-1,VARV,VARP
0,000000000,2017-12,10130000,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.00,0.00,-6000.00,-inf
1,000000000,2017-12,10611000,RESERVE LEGALE,-600.00,0.00,-600.00,-inf
2,000000000,2017-12,10688000,AUTRES RESERVES,-130673.92,0.00,-130673.92,-inf
3,000000000,2017-12,12000000,RÉSULTAT DE L'EXERCICE BÉNÉFIC,0.00,0.00,0.00,NaN
4,000000000,2017-12,20500000,"BREVETS, LICENCES, LOGICIELS..",4006.60,0.00,4006.60,inf
...,...,...,...,...,...,...,...,...
291,000000000,2018-12,77180000,AUTRES PROD EXCEP S/OP GESTION,-51.00,0.00,-51.00,-inf
292,000000000,2018-12,79100000,TRANSFERT DE CHARGES D'EXPLOIT,-46.07,0.00,-46.07,-inf
293,000000000,2018-12,79101000,RÉINT. UTILIS. PERSONNELLE,-1750.50,-1823.44,72.94,-0.040001
294,000000000,2017-12,10999999,RESULTAT,-81496.79,0.00,-81496.79,-inf


In [11]:
db_cat = db_var.copy()

# Calcul des rubriques niveau 2
def rubrique_N2(row): 
        numero_compte = str(row["COMPTE_NUM"])
        value = float(row["VALUE"])
        
# BILAN SIMPLIFIE type IFRS NIV2

        to_check = ["^10", "^11", "^12", "^13", "^14"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "CAPITAUX_PROPRES"
        
        to_check = ["^15", "^16", "^17", "^18", "^19"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "DETTES_FINANCIERES"
        
        to_check = ["^20", "^21", "^22", "^23", "^25", "^26", "^27", "^28", "^29"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "IMMOBILISATIONS"
        
        to_check = ["^31", "^32", "^33", "^34", "^35", "^36", "^37", "^38", "^39"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "STOCKS"
        
        to_check = ["^40"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "DETTES_FOURNISSEURS"
        
        to_check = ["^41"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "CREANCES_CLIENTS"
        
        to_check = ["^42", "^43", "^44", "^45", "^46", "^47", "^48", "^49"]
        if any (re.search(x,numero_compte) for x in to_check):
            if value > 0:
                return "AUTRES_CREANCES"
            else:
                return "AUTRES_DETTES"
        
        to_check = ["^50", "^51", "^52", "^53", "^54", "^58", "^59"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "DISPONIBILITES"

        
# COMPTE DE RESULTAT DETAILLE NIV2

        to_check = ["^60"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "ACHATS"
        
        to_check= ["^61", "^62"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "SERVICES_EXTERIEURS"
        
        to_check = ["^63"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "TAXES"        
    
        to_check = ["^64"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "CHARGES_PERSONNEL"    
    
        to_check = ["^65"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "AUTRES_CHARGES"  
    
        to_check = ["^66"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "CHARGES_FINANCIERES"
        
        to_check = ["^67"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "CHARGES_EXCEPTIONNELLES"
        
        to_check = ["^68", "^78"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "AMORTISSEMENTS"
        
        to_check = ["^69"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "IMPOT"
        
        to_check = ["^70"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "VENTES"
        
        to_check = ["^71", "^72"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "PRODUCTION_STOCKEE_IMMOBILISEE"
        
        to_check = ["^74"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "SUBVENTIONS_D'EXPL."
        
        to_check = ["^75", "^791"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "AUTRES_PRODUITS_GESTION_COURANTE"
        
        to_check = ["^76", "^796"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "PRODUITS_FINANCIERS"
        
        to_check = ["^77", "^797"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "PRODUITS_EXCEPTIONNELS"
        
        to_check = ["^78"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "REPRISES_AMORT._DEP."

        to_check = ["^8"]
        if any (re.search(x,numero_compte) for x in to_check):
            return "COMPTES_SPECIAUX"

        
# Calcul des rubriques niveau 1
def rubrique_N1(row):
    categorisation = row.RUBRIQUE_N2
    
# BILAN SIMPLIFIE type IFRS N1

    to_check = ["CAPITAUX_PROPRES", "DETTES_FINANCIERES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PASSIF_NON_COURANT"
    
    to_check = ["IMMOBILISATIONS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "ACTIF_NON_COURANT"
    
    to_check = ["STOCKS", "CREANCES_CLIENTS", "AUTRES_CREANCES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "ACTIF_COURANT"   
        
    to_check = ["DETTES_FOURNISSEURS", "AUTRES_DETTES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PASSIF_COURANT"
    
    to_check = ["DISPONIBILITES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "DISPONIBILITES"

    
# COMPTE DE RESULTAT SIMPLIFIE N1
    
    to_check = ["ACHATS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "COUTS_DIRECTS"
    
    to_check = ["SERVICES_EXTERIEURS", "TAXES", "CHARGES_PERSONNEL", "AUTRES_CHARGES", "AMORTISSEMENTS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHARGES_EXPLOITATION"
    
    to_check = ["CHARGES_FINANCIERES"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHARGES_FINANCIERES"
        
    to_check = ["CHARGES_EXCEPTIONNELLES", "IMPOT"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHARGES_EXCEPTIONNELLES"        

    to_check = ["VENTES", "PRODUCTION_STOCKEE_IMMOBILISEE"]
    if any(re.search(x, categorisation) for x in to_check):
        return "CHIFFRE_D'AFFAIRES"
    
    to_check = ["SUBVENTIONS_D'EXPL.", "AUTRES_PRODUITS_GESTION_COURANTE", "REPRISES_AMORT._DEP."]
    if any(re.search(x, categorisation) for x in to_check):
        return "PRODUITS_EXPLOITATION"
    
    to_check = ["PRODUITS_FINANCIERS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PRODUITS_FINANCIERS"
        
    to_check = ["PRODUITS_EXCEPTIONNELS"]
    if any(re.search(x, categorisation) for x in to_check):
        return "PRODUITS_EXCEPTIONNELS"      
        

# Calcul des rubriques niveau 0
def rubrique_N0(row):
    masse = row.RUBRIQUE_N1
    
    to_check = ["ACTIF_NON_COURANT", "ACTIF_COURANT", "DISPONIBILITES"]
    if any(re.search(x, masse) for x in to_check):
        return "ACTIF"
    
    to_check = ["PASSIF_NON_COURANT", "PASSIF_COURANT"]
    if any(re.search(x, masse) for x in to_check):
        return "PASSIF"

    to_check = ["COUTS_DIRECTS", "CHARGES_EXPLOITATION", "CHARGES_FINANCIERES", "CHARGES_EXCEPTIONNELLES"]
    if any(re.search(x, masse) for x in to_check):
        return "CHARGES"   
        
    to_check = ["CHIFFRE_D'AFFAIRES", "PRODUITS_EXPLOITATION", "PRODUITS_FINANCIERS", "PRODUITS_EXCEPTIONNELS"]
    if any(re.search(x, masse) for x in to_check):
        return "PRODUITS"  
    
    
# Mapping des rubriques 
db_cat["RUBRIQUE_N2"] = db_cat.apply(lambda row: rubrique_N2(row), axis=1)
db_cat["RUBRIQUE_N1"] = db_cat.apply(lambda row: rubrique_N1(row), axis=1)
db_cat["RUBRIQUE_N0"] = db_cat.apply(lambda row: rubrique_N0(row), axis=1)    


# Reorganisation colonne
to_select = ['ENTITY',
             'PERIOD',
             'COMPTE_NUM', 
             'RUBRIQUE_N0',
             'RUBRIQUE_N1',
             'RUBRIQUE_N2',
             'RUBRIQUE_N3',
             'VALUE',
             'VALUE_N-1',
             'VARV',
             'VARP']
db_cat = db_cat[to_select]

db_cat

,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,RUBRIQUE_N3,VALUE,VALUE_N-1,VARV,VARP
0,000000000,2017-12,10130000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.00,0.00,-6000.00,-inf
1,000000000,2017-12,10611000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,RESERVE LEGALE,-600.00,0.00,-600.00,-inf
2,000000000,2017-12,10688000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,AUTRES RESERVES,-130673.92,0.00,-130673.92,-inf
3,000000000,2017-12,12000000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,RÉSULTAT DE L'EXERCICE BÉNÉFIC,0.00,0.00,0.00,NaN
4,000000000,2017-12,20500000,ACTIF,ACTIF_NON_COURANT,IMMOBILISATIONS,"BREVETS, LICENCES, LOGICIELS..",4006.60,0.00,4006.60,inf
...,...,...,...,...,...,...,...,...,...,...,...
291,000000000,2018-12,77180000,PRODUITS,PRODUITS_EXCEPTIONNELS,PRODUITS_EXCEPTIONNELS,AUTRES PROD EXCEP S/OP GESTION,-51.00,0.00,-51.00,-inf
292,000000000,2018-12,79100000,PRODUITS,PRODUITS_EXPLOITATION,AUTRES_PRODUITS_GESTION_COURANTE,TRANSFERT DE CHARGES D'EXPLOIT,-46.07,0.00,-46.07,-inf
293,000000000,2018-12,79101000,PRODUITS,PRODUITS_EXPLOITATION,AUTRES_PRODUITS_GESTION_COURANTE,RÉINT. UTILIS. PERSONNELLE,-1750.50,-1823.44,72.94,-0.040001
294,000000000,2017-12,10999999,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,RESULTAT,-81496.79,0.00,-81496.79,-inf


### Modèles de données des graphiques

#### REF_ENTITE

In [12]:
# Creation du dataset ref_entite
dataset_entite = db_cat.copy()

# Regrouper par entite
to_group = ["ENTITY"]
to_agg = {"ENTITY": "max"}
dataset_entite = dataset_entite.groupby(to_group, as_index=False).agg(to_agg)

# Affichage du modèle de donnée
dataset_entite

,ENTITY
0,000000000


#### REF_SCENARIO

In [13]:
# Creation du dataset ref_scenario
dataset_scenario = db_cat.copy()

# Regrouper par entite
to_group = ["PERIOD"]
to_agg = {"PERIOD": "max"}
dataset_scenario = dataset_scenario.groupby(to_group, as_index=False).agg(to_agg)

# Affichage du modèle de donnée
dataset_scenario

,PERIOD
0,2017-12
1,2018-12


#### KPIS

In [14]:
# Creation du dataset KPIS (CA, MARGE, EBE, BFR, CC, DF)
dataset_kpis = db_cat.copy()

# KPIs CA
dataset_kpis_ca = dataset_kpis[dataset_kpis.RUBRIQUE_N1.isin(["CHIFFRE_D'AFFAIRES"])]

to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N1"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ca = dataset_kpis_ca.groupby(to_group, as_index=False).agg(to_agg)

# Passage value postif
dataset_kpis_ca["VALUE"] = dataset_kpis_ca["VALUE"]*-1


# COUTS_DIRECTS
dataset_kpis_ha = dataset_kpis[dataset_kpis.RUBRIQUE_N1.isin(["COUTS_DIRECTS"])]

to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N1"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ha = dataset_kpis_ha.groupby(to_group, as_index=False).agg(to_agg)

# Passage value négatif
dataset_kpis_ha["VALUE"] = dataset_kpis_ha["VALUE"]*-1


# KPIs MARGE BRUTE (CA - COUTS DIRECTS)
dataset_kpis_mb = dataset_kpis_ca.copy()
dataset_kpis_mb = pd.concat([dataset_kpis_mb, dataset_kpis_ha], axis=0, sort=False)

to_group = ["ENTITY",
            "PERIOD"]
to_agg = {"VALUE": "sum"}

dataset_kpis_mb = dataset_kpis_mb.groupby(to_group, as_index=False).agg(to_agg)
dataset_kpis_mb["RUBRIQUE_N1"] = "MARGE"
dataset_kpis_mb = dataset_kpis_mb[["ENTITY", "PERIOD", "RUBRIQUE_N1", "VALUE"]]


# CHARGES EXTERNES
dataset_kpis_ce = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["SERVICES_EXTERIEURS"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ce = dataset_kpis_ce.groupby(to_group, as_index=False).agg(to_agg)

# Passage value negatif
dataset_kpis_ce["VALUE"] = dataset_kpis_ce["VALUE"]*-1


# IMPOTS
dataset_kpis_ip = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["TAXES"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ip = dataset_kpis_ip.groupby(to_group, as_index=False).agg(to_agg)

# Passage value negatif
dataset_kpis_ip["VALUE"] = dataset_kpis_ip["VALUE"]*-1


# CHARGES DE PERSONNEL
dataset_kpis_cp = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["CHARGES_PERSONNEL"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_cp = dataset_kpis_cp.groupby(to_group, as_index=False).agg(to_agg)

# Passage value negatif
dataset_kpis_cp["VALUE"] = dataset_kpis_cp["VALUE"]*-1


# AUTRES_CHARGES
dataset_kpis_ac = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["AUTRES_CHARGES"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ac = dataset_kpis_ac.groupby(to_group, as_index=False).agg(to_agg)

# Passage value negatif
dataset_kpis_ac["VALUE"] = dataset_kpis_ac["VALUE"]*-1


# SUBVENTIONS D'EXPLOITATION
dataset_kpis_ac = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["SUBVENTIONS_D'EXPL."])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_ac = dataset_kpis_ac.groupby(to_group, as_index=False).agg(to_agg)


# KPIs EBE = MARGE - CHARGES EXTERNES - TAXES - CHARGES PERSONNEL - AUTRES CHARGES + SUBVENTION D'EXPLOITATION
dataset_kpis_ebe = dataset_kpis_mb.copy()
dataset_kpis_ebe = pd.concat([dataset_kpis_ebe, dataset_kpis_ce, dataset_kpis_ip, dataset_kpis_cp, dataset_kpis_ac], axis=0, sort=False)

to_group = ["ENTITY", "PERIOD"]
to_agg = {"VALUE": "sum"}

dataset_kpis_ebe = dataset_kpis_ebe.groupby(to_group, as_index=False).agg(to_agg)
dataset_kpis_ebe["RUBRIQUE_N1"] = "EBE"
dataset_kpis_ebe = dataset_kpis_ebe[["ENTITY", "PERIOD", "RUBRIQUE_N1", "VALUE"]]


# KPIs CREANCES CLIENTS
dataset_kpis_cc = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["CREANCES_CLIENTS"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_cc = dataset_kpis_cc.groupby(to_group, as_index=False).agg(to_agg)

# Renommage colonne
to_rename = {'RUBRIQUE_N2': "RUBRIQUE_N1"}
dataset_kpis_cc = dataset_kpis_cc.rename(columns=to_rename)


# KPIs STOCKS
dataset_kpis_st = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["STOCKS"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_st = dataset_kpis_st.groupby(to_group, as_index=False).agg(to_agg)

# Renommage colonne
to_rename = {'RUBRIQUE_N2': "RUBRIQUE_N1"}
dataset_kpis_st = dataset_kpis_st.rename(columns=to_rename)


# KPIs DETTES FOURNISSEURS
dataset_kpis_df = dataset_kpis[dataset_kpis.RUBRIQUE_N2.isin(["DETTES_FOURNISSEURS"])]
to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_kpis_df = dataset_kpis_df.groupby(to_group, as_index=False).agg(to_agg)

# Renommage colonne
to_rename = {'RUBRIQUE_N2': "RUBRIQUE_N1"}
dataset_kpis_df = dataset_kpis_df.rename(columns=to_rename)

# Passage value positif
dataset_kpis_df["VALUE"] = dataset_kpis_df["VALUE"].abs()


# KPIs BFR = CREANCES + STOCKS - DETTES FOURNISSEURS
dataset_kpis_bfr_df = dataset_kpis_df.copy()

# Passage dette fournisseur value négatif
dataset_kpis_bfr_df["VALUE"] = dataset_kpis_bfr_df["VALUE"]*-1

dataset_kpis_bfr_df = pd.concat([dataset_kpis_cc, dataset_kpis_st, dataset_kpis_bfr_df], axis=0, sort=False)

to_group = ["ENTITY", "PERIOD"]
to_agg = {"VALUE": "sum"}
dataset_kpis_bfr_df = dataset_kpis_bfr_df.groupby(to_group, as_index=False).agg(to_agg)

# Creation colonne Rubrique_N1 = BFR
dataset_kpis_bfr_df["RUBRIQUE_N1"] = "BFR"

# Reorganisation colonne
dataset_kpis_bfr_df = dataset_kpis_bfr_df[["ENTITY", "PERIOD", "RUBRIQUE_N1", "VALUE"]]


# Creation du dataset final
dataset_kpis_final = pd.concat([dataset_kpis_ca, dataset_kpis_mb, dataset_kpis_ebe, dataset_kpis_cc, dataset_kpis_st, dataset_kpis_df, dataset_kpis_bfr_df], axis=0, sort=False)


# Creation colonne COMP
dataset_kpis_final['PERIOD_COMP'] = (dataset_kpis_final['PERIOD'].str[:4].astype(int) - 1).astype(str) + dataset_kpis_final['PERIOD'].str[-3:]
dataset_kpis_final

,ENTITY,PERIOD,RUBRIQUE_N1,VALUE,PERIOD_COMP
0,000000000,2017-12,CHIFFRE_D'AFFAIRES,549403.83,2016-12
1,000000000,2018-12,CHIFFRE_D'AFFAIRES,551927.22,2017-12
0,000000000,2017-12,MARGE,461703.68,2016-12
1,000000000,2018-12,MARGE,462069.12,2017-12
0,000000000,2017-12,EBE,111980.52,2016-12
1,000000000,2018-12,EBE,40784.01,2017-12
0,000000000,2017-12,CREANCES_CLIENTS,38959.55,2016-12
1,000000000,2018-12,CREANCES_CLIENTS,42575.63,2017-12
0,000000000,2017-12,STOCKS,4030.77,2016-12
1,000000000,2018-12,STOCKS,4134.75,2017-12


In [15]:
# creation base comparable pour dataset_kpis
dataset_kpis_final_comp = dataset_kpis_final.copy()

# Suppression de la colonne période
dataset_kpis_final_comp = dataset_kpis_final_comp.drop("PERIOD_COMP", axis=1)

# Renommage des colonnes
to_rename = {'VALUE': "VALUE_N-1", 
             'PERIOD': "PERIOD_COMP"}
dataset_kpis_final_comp = dataset_kpis_final_comp.rename(columns=to_rename)
dataset_kpis_final_comp

,ENTITY,PERIOD_COMP,RUBRIQUE_N1,VALUE_N-1
0,000000000,2017-12,CHIFFRE_D'AFFAIRES,549403.83
1,000000000,2018-12,CHIFFRE_D'AFFAIRES,551927.22
0,000000000,2017-12,MARGE,461703.68
1,000000000,2018-12,MARGE,462069.12
0,000000000,2017-12,EBE,111980.52
1,000000000,2018-12,EBE,40784.01
0,000000000,2017-12,CREANCES_CLIENTS,38959.55
1,000000000,2018-12,CREANCES_CLIENTS,42575.63
0,000000000,2017-12,STOCKS,4030.77
1,000000000,2018-12,STOCKS,4134.75


In [16]:
# Jointure entre les 2 tables dataset_kpis_final et dataset_kpis_vf
join_on = ["ENTITY",
           "PERIOD_COMP",
           "RUBRIQUE_N1"]
dataset_kpis_final = pd.merge(dataset_kpis_final, dataset_kpis_final_comp, how='left', on=join_on).drop("PERIOD_COMP", axis=1).fillna(0)

#Création colonne Var V
dataset_kpis_final["VARV"] = dataset_kpis_final["VALUE"] - dataset_kpis_final["VALUE_N-1"]

#Création colonne Var P (%)
dataset_kpis_final["VARP"] = dataset_kpis_final["VARV"] / dataset_kpis_final["VALUE_N-1"]

dataset_kpis_final

,ENTITY,PERIOD,RUBRIQUE_N1,VALUE,VALUE_N-1,VARV,VARP
0,000000000,2017-12,CHIFFRE_D'AFFAIRES,549403.83,0.00,549403.83,inf
1,000000000,2018-12,CHIFFRE_D'AFFAIRES,551927.22,549403.83,2523.39,0.004593
2,000000000,2017-12,MARGE,461703.68,0.00,461703.68,inf
3,000000000,2018-12,MARGE,462069.12,461703.68,365.44,0.000792
4,000000000,2017-12,EBE,111980.52,0.00,111980.52,inf
5,000000000,2018-12,EBE,40784.01,111980.52,-71196.51,-0.635794
6,000000000,2017-12,CREANCES_CLIENTS,38959.55,0.00,38959.55,inf
7,000000000,2018-12,CREANCES_CLIENTS,42575.63,38959.55,3616.08,0.092816
8,000000000,2017-12,STOCKS,4030.77,0.00,4030.77,inf
9,000000000,2018-12,STOCKS,4134.75,4030.77,103.98,0.025797


#### EVOLUTION CA

In [17]:
# Creation du dataset evol_ca
dataset_evol_ca = db_enr.copy()

# Filtre COMPTE_NUM = Chiffre d'Affaire (RUBRIQUE N1)
dataset_evol_ca = dataset_evol_ca[dataset_evol_ca['COMPTE_NUM'].str.contains(r'^70|^71|^72')]

# Regroupement 
to_group = ["ENTITY",
            "PERIOD",
            "MONTH",
            "MONTH_INDEX",
            "RUBRIQUE_N3"]
to_agg = {"VALUE": "sum"}
dataset_evol_ca = dataset_evol_ca.groupby(to_group, as_index=False).agg(to_agg)

dataset_evol_ca["VALUE"] = dataset_evol_ca["VALUE"].abs()


# Calcul de la somme cumulée
dataset_evol_ca = dataset_evol_ca.sort_values(by=["ENTITY", 'PERIOD', 'MONTH_INDEX']).reset_index(drop=True)
dataset_evol_ca['MONTH_INDEX'] = pd.to_datetime(dataset_evol_ca['MONTH_INDEX'], format="%m").dt.strftime("%m")
dataset_evol_ca['VALUE_CUM'] = dataset_evol_ca.groupby(["ENTITY", "PERIOD"], as_index=True).agg({"VALUE": "cumsum"})

# Affichage du modèle de donnée
dataset_evol_ca

,ENTITY,PERIOD,MONTH,MONTH_INDEX,RUBRIQUE_N3,VALUE,VALUE_CUM
0,000000000,2017-12,Jan,01,PRODUITS FINIS EXONÉRÉS,42550.15,42550.15
1,000000000,2017-12,Feb,02,PRODUITS FINIS EXONÉRÉS,40916.60,83466.75
2,000000000,2017-12,Mar,03,PRODUITS FINIS EXONÉRÉS,64832.82,148299.57
3,000000000,2017-12,Apr,04,PRODUITS FINIS EXONÉRÉS,48396.90,196696.47
4,000000000,2017-12,May,05,PRODUITS FINIS EXONÉRÉS,44719.45,241415.92
5,000000000,2017-12,Jun,06,PRODUITS FINIS EXONÉRÉS,59904.10,301320.02
6,000000000,2017-12,Jul,07,PRODUITS FINIS EXONÉRÉS,49540.65,350860.67
7,000000000,2017-12,Sep,09,PRODUITS FINIS EXONÉRÉS,48302.80,399163.47
8,000000000,2017-12,Oct,10,PRODUITS FINIS EXONÉRÉS,55081.15,454244.62
9,000000000,2017-12,Nov,11,PRODUITS FINIS EXONÉRÉS,44492.10,498736.72


#### CHARGES

In [18]:
#Creation du dataset charges
dataset_charges = db_cat.copy()

# Filtre RUBRIQUE_N0 = CHARGES
dataset_charges = dataset_charges[dataset_charges["RUBRIQUE_N0"] == "CHARGES"]

# Mettre en valeur positive VALUE
dataset_charges["VALUE"] = dataset_charges["VALUE"].abs()

# Affichage du modèle de donnée
dataset_charges

,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,RUBRIQUE_N3,VALUE,VALUE_N-1,VARV,VARP
51,000000000,2017-12,60180000,CHARGES,COUTS_DIRECTS,ACHATS,ACHATS MATIERES 1ERES FOURNIT.,27973.58,0.00,27973.58,inf
52,000000000,2017-12,60310000,CHARGES,COUTS_DIRECTS,ACHATS,STOCKS I ACHATS MAT. 1ERES,5054.62,0.00,5054.62,inf
53,000000000,2017-12,60315000,CHARGES,COUTS_DIRECTS,ACHATS,STOCKS II ACHATS MAT.1ERES,4030.77,0.00,-4030.77,-inf
54,000000000,2017-12,60480000,CHARGES,COUTS_DIRECTS,ACHATS,ACHATS PRESTATIONS EXONÉRÉS,48960.36,0.00,48960.36,inf
55,000000000,2017-12,60612000,CHARGES,COUTS_DIRECTS,ACHATS,ELECTRICITÉ,1684.63,0.00,1684.63,inf
...,...,...,...,...,...,...,...,...,...,...,...
283,000000000,2018-12,64900000,CHARGES,CHARGES_EXPLOITATION,CHARGES_PERSONNEL,C.I.C.E.,5832.00,-7020.00,1188.00,-0.169231
284,000000000,2018-12,65110000,CHARGES,CHARGES_EXPLOITATION,AUTRES_CHARGES,"REDEVANCES (MARQUES, CONCESSIO",250.00,0.00,250.00,inf
285,000000000,2018-12,66800000,CHARGES,CHARGES_FINANCIERES,CHARGES_FINANCIERES,AUTRES CHARGES FINANCIERES,0.07,275.22,-275.15,-0.999746
286,000000000,2018-12,68112000,CHARGES,CHARGES_EXPLOITATION,AMORTISSEMENTS,DOTATION AMORT / IMMO CORPOREL,4221.27,6042.08,-1820.81,-0.301355


#### POSITIONS TRESORERIE

In [19]:
# Creation du dataset trésorerie
dataset_treso = db_enr.copy()

# Filtre RUBRIQUE_N1 = TRESORERIE
dataset_treso = dataset_treso[dataset_treso['COMPTE_NUM'].str.contains(r'^5')].reset_index(drop=True)

# Cash in / Cash out ?
dataset_treso.loc[dataset_treso.VALUE > 0, "CASH_IN"] = dataset_treso.VALUE
dataset_treso.loc[dataset_treso.VALUE < 0, "CASH_OUT"] = dataset_treso.VALUE

# Regroupement 
to_group = ["ENTITY",
            "PERIOD",
            "MONTH",
            "MONTH_INDEX"]
to_agg = {"VALUE": "sum",
          "CASH_IN": "sum",
          "CASH_OUT": "sum"}
dataset_treso = dataset_treso.groupby(to_group, as_index = False).agg(to_agg).fillna(0)

# Cumul par période
dataset_treso = dataset_treso.sort_values(["ENTITY", "PERIOD", "MONTH_INDEX"])
dataset_treso['MONTH_INDEX'] = pd.to_datetime(dataset_treso['MONTH_INDEX'], format="%m").dt.strftime("%m")
dataset_treso['VALUE_LINE'] = dataset_treso.groupby(["ENTITY", 'PERIOD'], as_index=True).agg({"VALUE": "cumsum"})

# Mettre en valeur positive CASH_OUT
dataset_treso["CASH_OUT"] = dataset_treso["CASH_OUT"].abs()

# Affichage du modèle de donnée
dataset_treso

,ENTITY,PERIOD,MONTH,MONTH_INDEX,VALUE,CASH_IN,CASH_OUT,VALUE_LINE
4,000000000,2017-12,Jan,01,129503.90,168471.60,38967.70,129503.90
3,000000000,2017-12,Feb,02,-6594.49,42205.15,48799.64,122909.41
7,000000000,2017-12,Mar,03,267.46,41521.46,41254.00,123176.87
0,000000000,2017-12,Apr,04,-60412.14,65481.71,125893.85,62764.73
8,000000000,2017-12,May,05,7139.11,119644.49,112505.38,69903.84
6,000000000,2017-12,Jun,06,26363.71,44719.45,18355.74,96267.55
5,000000000,2017-12,Jul,07,16816.88,172363.85,155546.97,113084.43
1,000000000,2017-12,Aug,08,170.32,23093.20,22922.88,113254.75
11,000000000,2017-12,Sep,09,-3680.61,14201.45,17882.06,109574.14
10,000000000,2017-12,Oct,10,-3668.05,43058.55,46726.60,105906.09


#### BILAN

In [20]:
# Creation du dataset Bilan
dataset_bilan = db_cat.copy()

# Filtre RUBRIQUE_N0 = ACTIF & PASSIF
dataset_bilan = dataset_bilan[(dataset_bilan["RUBRIQUE_N0"].isin(["ACTIF", "PASSIF"]))]

# Regroupement R0/R1/R2
to_group = ["ENTITY",
            "PERIOD",
            "RUBRIQUE_N0",
            "RUBRIQUE_N1",
            "RUBRIQUE_N2"]
to_agg = {"VALUE": "sum"}
dataset_bilan = dataset_bilan.groupby(to_group, as_index = False).agg(to_agg).fillna(0)


# Mettre en valeur positive VALUE
dataset_bilan["VALUE"] = dataset_bilan["VALUE"].abs()

# Selectionner les colonnes
to_select = ["ENTITY",
             "PERIOD",
             "RUBRIQUE_N0",
             "RUBRIQUE_N1",
             "RUBRIQUE_N2",
             "VALUE"]
dataset_bilan = dataset_bilan[to_select]

# Affichage du modèle de donnée
dataset_bilan

,ENTITY,PERIOD,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,VALUE
0,000000000,2017-12,ACTIF,ACTIF_COURANT,AUTRES_CREANCES,25148.74
1,000000000,2017-12,ACTIF,ACTIF_COURANT,CREANCES_CLIENTS,38959.55
2,000000000,2017-12,ACTIF,ACTIF_COURANT,STOCKS,4030.77
3,000000000,2017-12,ACTIF,ACTIF_NON_COURANT,IMMOBILISATIONS,79527.73
4,000000000,2017-12,ACTIF,DISPONIBILITES,DISPONIBILITES,129172.30
5,000000000,2017-12,PASSIF,PASSIF_COURANT,AUTRES_DETTES,56471.60
6,000000000,2017-12,PASSIF,PASSIF_COURANT,DETTES_FOURNISSEURS,1596.78
7,000000000,2017-12,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,218770.71
8,000000000,2018-12,ACTIF,ACTIF_COURANT,AUTRES_CREANCES,26039.50
9,000000000,2018-12,ACTIF,ACTIF_COURANT,CREANCES_CLIENTS,42575.63


## Output

### Sauvegarde des fichiers en csv

In [21]:
def df_to_csv(df, filename):
    # Sauvegarde en csv
    df.to_csv(filename,
              sep=";",
              decimal=",",
              index=False)
    
    # Création du lien url
    naas_link = naas.asset.add(filename)
    
    # Création de la ligne
    data = {
        "OBJET": filename,
        "URL": naas_link,
        "DATE_EXTRACT": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    return pd.DataFrame([data])

In [22]:
dataset_logo = {
    "OBJET": "Logo",
    "URL": LOGO,
    "DATE_EXTRACT": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}
logo = pd.DataFrame([dataset_logo])
logo

,OBJET,URL,DATE_EXTRACT
0,Logo,https://landen.imgix.net/e5hx7wyzf53f/assets/2...,2021-06-03 13:06:21


In [23]:
import json

color = {"name":"Color",
         "dataColors":[COLOR_1, COLOR_2]}

with open("color.json", "w") as write_file:
    json.dump(color, write_file)

dataset_color = {
    "OBJET": "Color",
    "URL": naas.asset.add("color.json"),
    "DATE_EXTRACT": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}
pbi_color = pd.DataFrame([dataset_color])
pbi_color

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


,OBJET,URL,DATE_EXTRACT
0,Color,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:21


In [24]:
entite = df_to_csv(dataset_entite, "dataset_entite.csv")
entite

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


,OBJET,URL,DATE_EXTRACT
0,dataset_entite.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:21


In [25]:
scenario = df_to_csv(dataset_scenario, "dataset_scenario.csv")
scenario

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


,OBJET,URL,DATE_EXTRACT
0,dataset_scenario.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:22


In [26]:
kpis = df_to_csv(dataset_kpis_final, "dataset_kpis_final.csv")
kpis

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


,OBJET,URL,DATE_EXTRACT
0,dataset_kpis_final.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:22


In [27]:
evol_ca = df_to_csv(dataset_evol_ca, "dataset_evol_ca.csv")
evol_ca

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


,OBJET,URL,DATE_EXTRACT
0,dataset_evol_ca.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:22


In [28]:
charges = df_to_csv(dataset_charges, "dataset_charges.csv")
charges

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


,OBJET,URL,DATE_EXTRACT
0,dataset_charges.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:22


In [29]:
treso = df_to_csv(dataset_treso, "dataset_treso.csv")
treso

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


,OBJET,URL,DATE_EXTRACT
0,dataset_treso.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:23


In [30]:
bilan = df_to_csv(dataset_bilan, "dataset_bilan.csv")
bilan

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


,OBJET,URL,DATE_EXTRACT
0,dataset_bilan.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:23


### Création du fichier à intégrer dans PowerBI

In [31]:
db_powerbi = pd.concat([logo, pbi_color, entite, scenario, kpis, evol_ca, charges, treso, bilan], axis=0)
db_powerbi

,OBJET,URL,DATE_EXTRACT
0,Logo,https://landen.imgix.net/e5hx7wyzf53f/assets/2...,2021-06-03 13:06:21
0,Color,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:21
0,dataset_entite.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:21
0,dataset_scenario.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:22
0,dataset_kpis_final.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:22
0,dataset_evol_ca.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:22
0,dataset_charges.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:22
0,dataset_treso.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:23
0,dataset_bilan.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:23


In [32]:
df_to_csv(db_powerbi, "powerbi.csv")

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


,OBJET,URL,DATE_EXTRACT
0,powerbi.csv,https://public.naas.ai/YS0yRXN0ZXZlbnMtNDBob3R...,2021-06-03 13:06:23
